In [8]:
print(settings2.TABLE_NAME)

adidas


In [7]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings2 # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings2.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [9]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [10]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="iembd2019",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings2.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings2.TABLE_NAME, settings2.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    



In [11]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOEKN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [12]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings2.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

RT @UofLSheriff50: Last week the NCAA moved to allow 3rd party payments to student-athletes. 

This week the NCAA is issuing a notice of al…
Long: None, Lati: None
RT @LetItFlyTom: adidas might make up for it tho https://t.co/wRJA5forkP
Long: None, Lati: None
RT @Kameron_Hay: Adidas fumbled Jordan, Kobe, and LeBron but hey at least they have Kanye 🤒
Long: None, Lati: None
@uhnseenfhenian @AvengingAngel10 @Record_Sport Yep Adidas are 1 sells a hoodie for £50 out a jd other 1 sells them… https://t.co/emmvzvMMsu
Long: None, Lati: None
As promised! Go to our Instagram profile @dcaveofficial right now for an exclusive look at Part II of the D-Cave Ta… https://t.co/rS647nCEMK
Long: None, Lati: None
RT @DragonflyJonez: Adidas blew it with MJ and Bron Bron. Tough scene.
Long: None, Lati: None
Has it sank into everyone’s mind that @adidas didn’t want to pay MJ the extra $150k. $150k kept @adidas from billio… https://t.co/sqiI5dzyuF
Long: None, Lati: None
nike en brrrr
Long: None, Lati: None
RT 

RT @TheFashionLaw: And speaking of J. Crew, this trademark clash (initiated by adidas) is still underway ... https://t.co/LVV5fgOTQx https:…
Long: None, Lati: None
RT @the_ironsheik: ADIDAS ALL DAY I DREAM ABOUT SUPLEXING
Long: None, Lati: None
@TripleEntendre Sure I had some UA and Adidas sprinkled in. But out of 10 shoes I bought, 8 were Nikes. I imagine that’s pretty common.
Long: None, Lati: None
#Adidas #KanyeWest #Yeezy: ADIDAS YEEZY SLIDE RESIN KANYE ADILETTE FX0494 - US 11 - NEW IN BOX… https://t.co/RDZDcTew5c
Long: None, Lati: None
RT @CourtsideFilms: Jaylen Hands vs Kobi Simmons at the 2015 Adidas Gauntlet Finale! https://t.co/UaZXyKIxQk
Long: None, Lati: None
Business is Business. It’s strictly financial.
Long: None, Lati: None
RT @BOOSTLINKS: Ad: Ends tomorrow at 10AM ET
adidas NMD_R1 V2
$63 + shipping, retail $130
use code SPRING30
=&gt; https://t.co/ooa7O7oViE http…
Long: None, Lati: None
RT @DAFTPUNKBEYOND: Daft Punk Appear in Adidas Star Wars Ad (2010) #MayThe4thBeWithY

Adidas x 424 Overdye Cap - Black/Red
https://t.co/uqPUduXG6j https://t.co/QjyKLMxWuO
Long: None, Lati: None
What if Michael Jordan signed with Adidas instead of Nike? https://t.co/Xb9VyMdLs0
Long: None, Lati: None
Adidas x 424 Vocal Hoodie - Black/Red
https://t.co/rSzV6FMKpL https://t.co/Ix85TJUtm9
Long: None, Lati: None
Adidas x 424 Track Top - Black/Red
https://t.co/OCInDRLpYr https://t.co/NXP63YiqBP
Long: None, Lati: None
Adidas x 424 Track Pant - Black/Red
https://t.co/cjPDrGeLlq https://t.co/m90iAy3DdZ
Long: None, Lati: None
Adidas didn't want to sign young Michael Jordan because executives thought he was too short https://t.co/gRCdl5hMJH via @businessinsider
Long: None, Lati: None
RT @menscave7: Check out some awesome ATHLETIC shorts Nike Adidas UA vintage + new https://t.co/XxKkfOldn5…
#springbreak2020 #nohighschool…
Long: None, Lati: None
RT @ColinCowherd: Adidas &amp; Converse had MJ and let him go.  What’s that, a 50 Billion dollar mistake ? Or more ?
Long: None, Lati: None
R

KeyboardInterrupt: 